<a href="https://colab.research.google.com/github/qusrud0113/PyTorch-tutorial/blob/main/2023_05_Pytorch_tutorial_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0.4.2 nn.Module로 구현하는 로지스틱 회귀

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as  optim

In [3]:
torch.manual_seed(1)

In [4]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
X_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

nn.Sequential은 nn.Module층을 쌓을 수 있도록 함. 즉 nn.Sequential은 최적화 함수와 시그모이드 함수 등을 연결시켜주는 역할

In [5]:
model = nn.Sequential(
    nn.Linear(2,1), #input_dim = 2, output_dim = 1
    nn.Sigmoid() # 출력은 Sigmoid함수 사용
)

In [6]:
model(X_train) #현재는 W와 b가 랜덤하게 배정된 상태라 의미 X

tensor([[0.4020],
        [0.4147],
        [0.6556],
        [0.5948],
        [0.6788],
        [0.8061]], grad_fn=<SigmoidBackward0>)

In [7]:
 optimizer = optim.SGD(model.parameters(), lr = 1)
 epochs = 1000
 for epoch in range(epochs+1):
    hx = model(X_train)
    cost = F.binary_cross_entropy(hx, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        pred = hx >= torch.FloatTensor([0.5])
        crr_pred = pred.float() == y_train
        acc = crr_pred.sum().item()/len(crr_pred)

        print(f'Epoch: {epoch}, cost: {round(cost.item(), 6)}, Accuracy: {round(acc*100,2)}')

Epoch: 0, cost: 0.539713, Accuracy: 83.33
Epoch: 100, cost: 0.134272, Accuracy: 100.0
Epoch: 200, cost: 0.080486, Accuracy: 100.0
Epoch: 300, cost: 0.05782, Accuracy: 100.0
Epoch: 400, cost: 0.045251, Accuracy: 100.0
Epoch: 500, cost: 0.037228, Accuracy: 100.0
Epoch: 600, cost: 0.031649, Accuracy: 100.0
Epoch: 700, cost: 0.027538, Accuracy: 100.0
Epoch: 800, cost: 0.024381, Accuracy: 100.0
Epoch: 900, cost: 0.021877, Accuracy: 100.0
Epoch: 1000, cost: 0.019843, Accuracy: 100.0


In [8]:
model(X_train) >= torch.FloatTensor([0.5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])

In [9]:
# after training weight and bias
print(list(model.parameters()))

[Parameter containing:
tensor([[3.2534, 1.5181]], requires_grad=True), Parameter containing:
tensor([-14.4839], requires_grad=True)]


## 0.4.3 클래스로 파이토치 모델 구현

In [16]:
class binaryclassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [17]:
torch.manual_seed(1)

In [18]:
model = binaryclassifier()

In [19]:
optimizer = optim.SGD(model.parameters(), lr = 1)
epochs = 1000
for epoch in range(epochs +1):
    hx = model(X_train)
    cost = F.binary_cross_entropy(hx, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch %100 == 0:
        pred = hx >= torch.FloatTensor([0.5])
        crr_pred = pred.float() == y_train
        acc = crr_pred.sum().item()/len(crr_pred)

        print(f'Epoch: {epoch}, cost: {round(cost.item(), 6)}, Accuracy: {round(acc*100,2)}')        

Epoch: 0, cost: 0.539713, Accuracy: 83.33
Epoch: 100, cost: 0.134272, Accuracy: 100.0
Epoch: 200, cost: 0.080486, Accuracy: 100.0
Epoch: 300, cost: 0.05782, Accuracy: 100.0
Epoch: 400, cost: 0.045251, Accuracy: 100.0
Epoch: 500, cost: 0.037228, Accuracy: 100.0
Epoch: 600, cost: 0.031649, Accuracy: 100.0
Epoch: 700, cost: 0.027538, Accuracy: 100.0
Epoch: 800, cost: 0.024381, Accuracy: 100.0
Epoch: 900, cost: 0.021877, Accuracy: 100.0
Epoch: 1000, cost: 0.019843, Accuracy: 100.0


# 0.5 소프트맥스 회귀(Softmax Regression)
## 0.5.1 원-핫 인코딩(One-Hot Encoding)
> 원 - 핫 인코딩

선택해야하는 선택지의 개수만큼의 차원을 가지면서, 각 선택지에 해당하는 인덱스의 원소에는 1, 나머지엔 0을 갖도록하는 표현 방법.

    예시)
    강아지 = [1, 0, 0]
    고양이 = [0, 1, 0]
    냉장고 = [0, 0, 1] 

> 원 - 핫 벡터의 무작위성

각 클래스 간의 관계가 균등하다는 이점이 존재함. 바나나, 토마토, 사과를 분류하는 문제가 있다고 생각하자. 라벨인코딩을 이용하면 각 클래스가 정수로 넘버링 된다.

    바나나 = 1
    토마토 = 2
    사과 = 3

 예를 들어 평가 방법이 MSE(Mean Squre Error)일 때 
 $$ \text{Loss function} = \frac{1}{n} \sum_{i=1}^{n}(y_{i} - \hat{y}_{i})
 ^{2}, $$ 실제값이 토마토일 때, 예측값이 바나나라면 제곱 오차는 $$(2-1)^2 = 1$$이 되고, 실제값이 사과일 때, 예측값이 바나나라면 제곱 오차는 $$(3-1)^2 = 4$$가 된다. 즉, 예측값이 바나나일때, 실제값이 토마토일때의 오차보다 사과일때의 오차가 크므로, 바나나가 토마토에 가깝다는 정보를 얻는 것과 같다. 
\
\
 이러한 인코딩 방식은 순서 정보가 도움이 되는 분류문제를 풀거나 회귀를 통한 분류문제를 풀 때 이점이 있다. 하지만 대부분의 다중 클래스 분류문제는 각 클래스간의 순서의 의미가 없기 때문에 라벨인코딩 보단 원-핫 인코딩을 통해 클래스간의 관계를 균등하게 두는 것이 좋다.

    바나나 = [1, 0, 0]
    토마토 = [0, 1, 0]
    사과 = [0, 0, 1] 
이렇게 균등 오차를 계산하게 되면
$$ ((0,1,0) - (1,0,0))^2 = (0-1)^2 + (0-1)^2 + (0-0)^2 = 2 $$
$$ ((0,1,0) - (0,0,1))^2 = (0-0)^2 + (1-0)^2 + (0-1)^2 = 2 $$
가 되어 모든 클래스간의 오차가 균등함을 보여준다.
 

## 0.5.2 소프트맥스 회귀 이해하기
소프트맥스 회귀를 통하여 다중 클래스 분류(Multi-Class Classification) 문제를 해결해보자.


>소프트맥스 함수

분류해야하는 클래스가 k개 일때, k차원의 벡터를 입력 받아 각 클래스에 대한 확률을 추정한다. k차원의 벡터에서 i번째 원소를 $z_{i}$, i번째 클래스가 정답일 확률을 $p_{i}$로 정의할 때, 소프트맥스 함수는 $p_{i}$를 다음과 같이 정의한다: $$p_{i} = \frac{e^{z_{i}}}{\sum_{j=1}^{k} e^{z_{j}}} \quad for \quad i=1,2,\dots, k$$

따라서 $z = (z_{1},\dots, z_{k})$일때, softmax function은 $$softmax(z) = \bigg(\frac{e^{z_{1}}}{\sum_{j=1}^{k} e^{z_{j}}},\dots, \frac{e^{z_{k}}}{\sum_{j=1}^{k} e^{z_{j}}}\bigg) = (p_{1},\dots,p_{k}), \quad where \quad \sum_{i=1}^{k}p_{i} = 1$$
이다.


> 크로스 엔트로피 함수

소프트맥스 회귀에서는 cost function을 크로스 엔트로피 함수를 사용한다:
    
    Define
    y : 실제값
    k : class 수
    p_j : j번째 클래스일 확률

$$ cost(W) = -\sum_{j=1}^{j} y_{j}ln(p_{j}) = -ln(p_{c}) \quad \text{if c-th index is solution}$$
이를 $n$개의 전체 데이터에 대한 평균으로 구한다면, 최종 비용 함수는 다음과 같다:
$$ cost(W) = -\frac{1}{n}\sum_{i=1}^{n}\sum_{j=1}^{k} y_{j}^{(i)}ln(p_{j}^{(i)}) $$



## 0.5.3 소프트맥스 회귀의 비용 함수 구현


In [20]:
import torch
import torch.nn.functional as F

In [21]:
torch.manual_seed(1)

In [22]:
z = torch.FloatTensor([1,2,3])

In [23]:
hx = F.softmax(z, dim = 0)
print(hx)

tensor([0.0900, 0.2447, 0.6652])


In [25]:
sum(hx)

tensor(1.)

In [27]:
z = torch.rand(3, 5 , requires_grad =True)

In [30]:
# dim 0,1은 softmax를 적용할 dim 위치임. 만약 dim이 1이면 열의 합이 1이 되게하는 softmax함수임
hx2 = F.softmax(z, dim = 1)
print(hx2)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [31]:
# torch.randint(low, high, size): low~high 사이에 정수를 뽑은 뒤 size 크기의 tensor 만들기
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [46]:
# one-hot encoding
# 3x5의 모든 원소가 0인 tensor 생성
y_one_hot = torch.zeros_like(hx2) 
# y.unsqueeze(1)은 1x3인 tensor를 3x1 tensor로 전환
# scatter(dim, value_index, apply_value)
# scatter함수를 통해 dim = 1(열) 중, y의 값을 위치로 변환하여, apply_value = 1로 변환.
# scatter 이후 '_'는 inplace operation임. 
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [53]:
# cost 함수 구현
cost = (y_one_hot*-torch.log(hx2)).sum(dim = 1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


실제 계산식
$$ cost(W) = -\frac{1}{3}\sum_{i=1}^{3}\sum_{j=1}^{k} y_{j}^{(i)}ln(p_{j}^{(i)}) = \frac{1}{3}(-1*ln(0.1855) - 1*ln(0.2322) - 1*ln(0.1986)) = 1.46884$$

In [54]:
# another method of cross_entropy
# using F.log_softmax()
(y_one_hot*-F.log_softmax(z, dim =1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [55]:
# not using one-hot encoding
F.nll_loss(F.log_softmax(z,dim = 1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

nll은 Negative Log Likelihood의 약자입니다.

In [56]:
# final method
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

F.cross_entropy는 cost함수와 softmax함수가 포함되어 있어 구현시 혼동이 가능할 수 있습니다.